# **Purpose: Collect News Article titles about a currency pair and get its associated price movement (Up or Down) **

****

In [6]:
!pip install beautifulsoup4 requests
# I made these changes bitch

**Given a currency pair, get relevant news articles from the previous day**

In [7]:
from bs4 import BeautifulSoup as bs
import pandas as pd
from urllib.request import Request, urlopen
from tqdm import tqdm

class fx_news_scraper():
    def __init__(self):
        self.number_of_pages = 29
        self.news_list = []
        self.target_url = 'https://www.investing.com/news/forex-news/'
        self.output_csv = 'app/data/fx-news-archive_test.csv'
        self.news_dataset = pd.DataFrame()

    def scraper(self):
        for page_no in tqdm(range(1, self.number_of_pages+1)):
            url = self.target_url + str(page_no)
            req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
            
            webpage = urlopen(req).read()
            
            soup = bs(webpage, "html.parser")
            
            news_container_div = soup.find('div', class_= 'news-analysis-v2_articles-container__3fFL8 mdMax:px-3 mb-12')

            try:
                for news_div in news_container_div.find_all('div', class_= 'news-analysis-v2_content__z0iLP w-full text-xs sm:flex-1'):
                    news_title_a = news_div.find('a', class_ = 'text-inv-blue-500 hover:text-inv-blue-500 hover:underline focus:text-inv-blue-500 focus:underline whitespace-normal text-sm font-bold leading-5 !text-[#181C21] sm:text-base sm:leading-6 lg:text-lg lg:leading-7')
                    news_body    = news_div.find('p')
                    news_date    = news_div.find('time', class_ = 'mx-1 shrink-0 text-xs leading-4').get('datetime')
                    
                    news_href    = news_title_a.get('href')
                    
                    self.news_list.append([page_no, news_date, news_title_a.text , news_body.text, news_href])

                    
            except ValueError as vEr:
                print(vEr)

            
            self.news_dataset = pd.DataFrame(self.news_list, columns = ['page', 'date','title','body', 'url'] )
            self.news_dataset.to_csv(self.output_csv)
            
            page_no += 1

        print ('scraping completed')
        print (f'output --> {self.output_csv}')

In [8]:
news_scraper = fx_news_scraper()
news_scraper.scraper()

  0%|          | 0/29 [00:00<?, ?it/s]


HTTPError: HTTP Error 403: Forbidden

In [65]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

def get_forex_news_five_years(currency_pair):

    headers = {"User-Agent": "Mozilla/5.0"}
    articles = []
    
    page_count = 0
    stop_flag = False
    article_titles = []
    article_dates = []
    while not stop_flag and page_count <= 300:
        if page_count == 0:
            url = 'https://www.investing.com/currencies/'+currency_pair+'-news'
        else:
            url = 'https://www.investing.com/currencies/'+currency_pair+'-news/'+str(page_count)
        response = requests.get(url, headers=headers)
        print('url:',url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            news_list = soup.find('ul', attrs={'data-test': 'news-list'})

            # Extract all <article> elements inside it
            articles = news_list.find_all('article')
            for article in articles:
                div_tag = article.find('div', class_='block w-full sm:flex-1')
                if div_tag:
                    # Extract the <a> tag inside the <div>
                    a_tag = div_tag.find('a', href=True)
                else:
                    print("coudln't find div tag of class block w-full sm:flex-1")
                    assert(False)
                if a_tag:
                    article_title = a_tag.get_text(strip=True)
                    
                else:
                    print("coudln't find a tag with href")
                    assert(False)

                time_tag = div_tag.find('time', datetime=True)

                if time_tag:
                    # Get the datetime attribute from the <time> tag
                    datetime_value = time_tag['datetime'].split()[0]
                    dt = datetime.strptime(datetime_value, '%Y-%m-%d').date()
                    current_date = datetime.now().date()
                    diff_in_years = current_date.year - dt.year
                    if diff_in_years == 5:
                        stop_flag = True
                    else:
                        article_titles.append(article_title)
                        article_dates.append(datetime_value)


                else:
                    print("No <time> tag with datetime attribute found.")
                    assert(False)

        else:
            print("couldn't get page info:",url)
            assert(False)

        page_count += 1

    df = pd.DataFrame({
    'Title': article_titles,
    'Date': article_dates
    })
    c1 = currency_pair.split('-')[0]
    c2 = currency_pair.split('-')[1]
    str_ = c1+'_'+c2
    df.to_csv('currency_pair_info'+str_+'.csv', index=False)
    print("saved csv")
    print("done")

currency_pair = 'gbp-usd'
pred_horizon = 5 # in days
get_forex_news_five_years(currency_pair, pred_horizon)


url: https://www.investing.com/currencies/gbp-usd-news
url: https://www.investing.com/currencies/gbp-usd-news/1
url: https://www.investing.com/currencies/gbp-usd-news/2
url: https://www.investing.com/currencies/gbp-usd-news/3
url: https://www.investing.com/currencies/gbp-usd-news/4
url: https://www.investing.com/currencies/gbp-usd-news/5
url: https://www.investing.com/currencies/gbp-usd-news/6
url: https://www.investing.com/currencies/gbp-usd-news/7
url: https://www.investing.com/currencies/gbp-usd-news/8
url: https://www.investing.com/currencies/gbp-usd-news/9
url: https://www.investing.com/currencies/gbp-usd-news/10
url: https://www.investing.com/currencies/gbp-usd-news/11
url: https://www.investing.com/currencies/gbp-usd-news/12
url: https://www.investing.com/currencies/gbp-usd-news/13
url: https://www.investing.com/currencies/gbp-usd-news/14
url: https://www.investing.com/currencies/gbp-usd-news/15
url: https://www.investing.com/currencies/gbp-usd-news/16
url: https://www.investing